In [ ]:
import cv2
import numpy as np

In [ ]:
input_video_path = "/home/roboticscorner/Python/people_with_helmet_working_1920_1080_50fps.mp4"



In [ ]:
! python3 detect.py --source inference-data/busy_street.mp4 --weights yolov7.pt --name video_1 --view-img

In [ ]:
#loading weights

net = cv2.dnn.readNetFromONNX("best.onnx")
net.setPreferableBackend(cv2.dnn.DNN_BACKEND_CUDA)  # or DNN_BACKEND_OPENCV
net.setPreferableTarget(cv2.dnn.DNN_TARGET_CUDA)    # or DNN_TARGET_CPU

In [ ]:
cap = cv2.VideoCapture("input_video.mp4")

In [ ]:
def compute_bound(frame, outputs):

    # Load your class names (update with your custom dataset names)
    class_names = ["head", "helmet", "person"]  # Replace with your own

    # NMS & draw config
    conf_threshold = 0.25
    nms_threshold = 0.45

    # Assuming `outputs` is the result from net.forward()
    # shape: [num_detections, 85]
    detections = outputs[0]  # usually the first output

    boxes = []
    confidences = []
    class_ids = []

    for detection in detections:
        scores = detection[5:]
        class_id = np.argmax(scores)
        confidence = detection[4] * scores[class_id]

        if confidence > conf_threshold:
            cx, cy, w, h = detection[0:4]
            x = int(cx - w / 2)
            y = int(cy - h / 2)
            boxes.append([x, y, int(w), int(h)])
            confidences.append(float(confidence))
            class_ids.append(class_id)

    # Apply NMS
    indices = cv2.dnn.NMSBoxes(boxes, confidences, conf_threshold, nms_threshold)

    # Draw boxes
    for i in indices:
        i = i[0] if isinstance(i, (tuple, list, np.ndarray)) else i
        box = boxes[i]
        x, y, w, h = box
        label = f"{class_names[class_ids[i]]}: {confidences[i]:.2f}"

        # Draw rectangle
        cv2.rectangle(frame, (x, y), (x + w, y + h), (0, 255, 0), 2)

        # Draw label
        cv2.putText(frame, label, (x, y - 10), cv2.FONT_HERSHEY_SIMPLEX,
                0.5, (0, 255, 0), 2)
    return frame


In [ ]:
while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    blob = cv2.dnn.blobFromImage(frame, 1/255.0, (640, 640), swapRB=True, crop=False)
    net.setInput(blob)
    outputs = net.forward()

    # TODO: Add NMS + draw boxes here
    frame = compute_bound(frame, outputs)

    cv2.imshow("YOLOv7 Detection", frame)
    if cv2.waitKey(1) == ord('q'):
        break

In [ ]:
cap.release()
cv2.destroyAllWindows()